# The data
The dataset is saved in multiple tables of a database:
You can find a full copy of the database "as of the start of the project" in the file `data/bank/start.db`.

We will extract the database into CSV files so that we can work with them a _bit_ more easily*.

This notebook transforms the initial database into two CSV files:

- `data/csv/bank_marketing_2008-05-01_to_2010-07-31.csv` (past: customers which have a label)
- `data/csv/bank_marketing_2010-08-01_to_2010-09-30.csv` (to infer: customers to predict in the campaign between said dates)

## Schema

**Here are the unique considerations/specifications for this class:**
1. `campaign` corresponds to the number of times it took an agent in order to reach.
2. A contact (not the column) means an _attempt to reach a customer_


|Variable|Also called|Description|Type|
|--------|-----------|-----------|----|
|age|||(numeric)||
|job||type of job| (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')|
|marital||marital status| (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)|
|education||| (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')|
|default||has credit in default?| (categorical: 'no','yes','unknown')|
|housing||has housing loan?| (categorical: 'no','yes','unknown')|
|loan||has personal loan?| (categorical: 'no','yes','unknown')|
|**RELATED WITH THE LAST CONTACT OF THE CURRENT CAMPAIGN:**|
|contact|comm_type|contact communication type| (categorical: 'cellular','telephone') |
|month|comm_month|last contact month of year| (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')|
|day_of_week|comm_day|last contact day of the week| (categorical: 'mon','tue','wed','thu','fri')|
|duration|comm_duration|last contact duration, in seconds| (numeric). Important note:  this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.|
|**OTHER ATTRIBUTES**|
|campaign|curr_n_contact|number of contacts performed during this campaign and for this client| (numeric, includes last contact)
|pdays|days_since_last_campaign|number of days that passed by after the client was last contacted from a previous campaign| (numeric; 999 means client was not previously contacted)
|previous|last_n_contact|number of contacts performed before this campaign and for this client| (numeric)
|poutcome|last_outcome|outcome of the previous marketing campaign| (categorical: 'failure','nonexistent','success')
|**SOCIAL AND ECONOMIC CONTEXT ATTRIBUTES**|
|emp.var.rate||employment variation rate - quarterly indicator| (numeric)|
|cons.price.idx||consumer price index - monthly indicator| (numeric)|
|cons.conf.idx||consumer confidence index - monthly indicator| (numeric)|
|euribor3m||euribor 3 month rate - daily indicator| (numeric)|
|y|curr_outcome|Whether deposited|(categorical: 'yes', 'no')|

## Notes
We are working with a **modified version** of Bank Marketing dataset. Notably, not all the data is there for didactic reasons that will soon come.


<hr>



In [ ]:
! pip install pandas

In [ ]:
import pandas as pd

In [ ]:
! pwd

In [ ]:
! ls data

In [ ]:
! ls -l data/bank

In [ ]:
! readlink -f data/bank/start.db

### Socio eco

In [ ]:
df_socioeco = pd.read_csv('./data/external/socio_economic_indices_data.csv', sep=';')

In [ ]:
df_socioeco.head()

## From database to CSVs

In [ ]:
import sqlite3

In [ ]:
c = sqlite3.connect('./data/bank/start.db')

In [ ]:
# Print all campaigns
cur = c.cursor()
cur.execute(
    "SELECT comm_date,comm_year,comm_month,comm_day,comm_type,curr_outcome,comm_duration,curr_n_contact,days_since_last_campaign,last_n_contact,last_outcome,customer_id \
                FROM campaignMissions")

rows = cur.fetchall()

print(len(rows))
print(rows[-3:])

cur.close()

In [ ]:
cur = c.execute("SELECT * FROM customers")
customer_cols = ['id', 'first_name', 'last_name', 'email', 'phone', 'age', 'job',
            'marital', 'education']
df_customers = pd.DataFrame(cur, columns=customer_cols)

In [ ]:
df_customers.head(2)

In [ ]:
# Print not done campaigns
cur = c.execute("SELECT * FROM sqlite_master WHERE type='table';")

cur.execute(
            "SELECT comm_date,comm_year,comm_month,comm_day,comm_type,curr_outcome,comm_duration,curr_n_contact,days_since_last_campaign,last_n_contact,last_outcome,customer_id\
                    FROM campaignMissions WHERE comm_duration IS NULL AND curr_outcome IS NULL"
        )
df_not_done =  pd.DataFrame(cur, columns=[
            'comm_date', 'comm_year', 'comm_month', 'comm_day', 'comm_type',
            'curr_outcome', 'comm_duration', 'curr_n_contact',
            'days_since_last_campaign', 'last_n_contact', 'last_outcome',
            'customer_id'
        ])

In [ ]:
# Get done campaigns
cur = c.execute(
    "SELECT comm_date,comm_year,comm_month,comm_day,comm_type,curr_outcome,comm_duration,curr_n_contact,days_since_last_campaign,last_n_contact,last_outcome,customer_id \
            FROM campaignMissions WHERE comm_duration IS NOT NULL AND curr_outcome IS NOT NULL"
)

df_done = pd.DataFrame(cur, columns=[
            'comm_date', 'comm_year', 'comm_month', 'comm_day', 'comm_type',
            'curr_outcome', 'comm_duration', 'curr_n_contact',
            'days_since_last_campaign', 'last_n_contact', 'last_outcome',
            'customer_id'
        ])

In [ ]:
df_done.head(2)

In [ ]:
df_not_done.head(2)

In [ ]:
cur = c.execute("SELECT status,start_date,due_date,amount_due,default_penalties,customer_id FROM mortgages")
df_mortgages = pd.DataFrame(cur, columns=["status","start_date","due_date","amount_due","default_penalties","customer_id"])

In [ ]:
df_mortgages.head()

In [ ]:
cur = c.execute("SELECT status,start_date,due_date,amount_due,default_penalties,customer_id FROM loans")
df_loans = pd.DataFrame(cur, columns=["status","start_date","due_date","amount_due","default_penalties","customer_id"])

<hr>

In [ ]:
df_mortgages.head(3)

In [ ]:
df_loans['default_penalties'] == df_mortgages['default_penalties']

### Merging all tables
#### DONE
We need to make a dataframe with merged info

In [ ]:
df_2 = pd.merge(df_customers, df_done, left_on='id', right_on='customer_id',)

In [ ]:
df_2 = pd.merge(df_2, df_loans, on='customer_id')

In [ ]:
df_2 = pd.merge(df_2, df_mortgages, on='customer_id')

In [ ]:
df_2 = pd.merge(df_2, df_socioeco, left_on='comm_date', right_on='date')

In [ ]:
df_2['loan'] = df_2['status_x'].replace({'paid': 'no','ongoing': 'yes', 'unknown': 'unknown'})
df_2['housing'] = df_2['status_y'].replace({'paid': 'no','ongoing': 'yes', 'unknown': 'unknown'})

In [ ]:
def make_penalty_indicator_x(row):
    if pd.isna(row['default_penalties_x']):
        return 'unknown'
    if row['default_penalties_x'] == 0:
        return 'no'
    elif row['default_penalties_x'] > 0:
        return 'yes'

def make_penalty_indicator_y(row):
    if pd.isna(row['default_penalties_y']):
        return 'unknown'
    if row['default_penalties_y'] == 0:
        return 'no'
    elif row['default_penalties_y'] > 0:
        return 'yes'

In [ ]:
df_2['default_x'] = df_2.apply(make_penalty_indicator_x, axis=1)
df_2['default_y'] = df_2.apply(make_penalty_indicator_y, axis=1)

In [ ]:
def merge(row):
    if row['default_x'] == 'yes' or row['default_y'] == 'yes':
        return 'yes'
    elif row['default_x'] == 'unknown' or row['default_y'] == 'unknown':
        return 'unknown'
    elif row['default_x'] == 'no' and row['default_y'] == 'no':
        return 'no'

In [ ]:
df_2['default'] = df_2.apply(merge, axis=1)

#### Not done

In [ ]:
df_3 = pd.merge(df_customers, df_not_done, left_on='id', right_on='customer_id',)

In [ ]:
df_3 = pd.merge(df_3, df_loans, on='customer_id')

In [ ]:
df_3 = pd.merge(df_3, df_mortgages, on='customer_id')

In [ ]:
df_3 = pd.merge(df_3, df_socioeco, left_on='comm_date', right_on='date')

In [ ]:
df_3['loan'] = df_3['status_x'].replace({'paid': 'no','ongoing': 'yes', 'unknown': 'unknown'})
df_3['housing'] = df_3['status_y'].replace({'paid': 'no','ongoing': 'yes', 'unknown': 'unknown'})

In [ ]:
def make_penalty_indicator_x(row):
    if pd.isna(row['default_penalties_x']):
        return 'unknown'
    if row['default_penalties_x'] == 0:
        return 'no'
    elif row['default_penalties_x'] > 0:
        return 'yes'

def make_penalty_indicator_y(row):
    if pd.isna(row['default_penalties_y']):
        return 'unknown'
    if row['default_penalties_y'] == 0:
        return 'no'
    elif row['default_penalties_y'] > 0:
        return 'yes'

In [ ]:
df_3['default_x'] = df_3.apply(make_penalty_indicator_x, axis=1)
df_3['default_y'] = df_3.apply(make_penalty_indicator_y, axis=1)

In [ ]:
df_3['default'] = df_3.apply(merge, axis=1)

TO DROP

In [ ]:
TO_DROP_COLS = ["id","customer_id","first_name","last_name","email","phone", "date", "comm_year", "comm_date","status_x","start_date_x","due_date_x","amount_due_x","default_penalties_x","status_y","start_date_y","due_date_y","amount_due_y","default_penalties_y", "default_x", "default_y"]

In [ ]:
COL_ORDER = ['age', 'job', 'marital', 'education', 'comm_month',
       'comm_day', 'comm_type', 'comm_duration', 'curr_n_contact', 'days_since_last_campaign', 'last_n_contact',
       'last_outcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'housing', 'loan',
       'default', 'curr_outcome']

In [ ]:
df_final_not_done = df_3.drop(columns=TO_DROP_COLS)

In [ ]:
df_final = df_2.drop(columns=TO_DROP_COLS)

In [ ]:
print(df_final.shape)
print(df_final_not_done.shape)

In [ ]:
# Saving dataframes
df_final.to_csv('./data/csv/bank_marketing_2008-05-01_to_2010-07-31.csv', sep=';', index=False)
df_final_not_done.to_csv('./data/csv/bank_marketing_2010-08-01_to_2010-09-30.csv', sep=';', index=False)

In [ ]:
df_final.columns

## From CSVs to start of EDA process

In [ ]:
df1 = pd.read_csv('./data/csv/bank_marketing_2008-05-01_to_2010-07-31.csv', sep=';')

In [ ]:
df2 = pd.read_csv('./data/csv/bank_marketing_2010-08-01_to_2010-09-30.csv', sep=';')

In [ ]:
print("df1", df1.shape)
print("df2", df2.shape)